<a href="https://colab.research.google.com/github/BrotherNgo07/cs4372/blob/main/Recommendation_System_Assignment_SOLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation Systems
We will use the surprise library of Python. Details are available at: http://surpriselib.com

We will first work through an example using a built-in dataset and then use a custom one.

First, ensure that you have the library installed and then load the required packages.

In [1]:
!pip uninstall -y numpy
!pip install "numpy==1.26.4"

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 111.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.2

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 12.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2555155 sha256=c189dfb317dbbc7e1b4f2de88be952c58892efa71bfaab38216e1d9abc147cae
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [2]:
import io

import numpy as np
import pandas as pd
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir
from surprise import accuracy
from surprise.model_selection import KFold

For a recommendation system, we require a file containing at least 3 things - userId, itemId, and rating. Any other information is not needed, but can be good for human analysis of results.

Let's load the built in ml-100k dataset that contains movies and ratings.

In [3]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [4]:
# Let's see what files come with the dataset
!ls /root/.surprise_data/ml-100k/ml-100k/

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item


In [5]:
# TODO: Show the first 10 lines of the u.data, and u.item files
!head -10 /root/.surprise_data/ml-100k/ml-100k/u.data

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


## Algorithms
Let's look at some of the algorithms available with the package

In [6]:
?KNNBaseline

The nearest neighbor methods works by searching for neighbors using the utility matrix. Let's create a nearest neighbor first by item and user

In [7]:
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
# we are going to use item-item similarity
sim_options = {'name': 'pearson_baseline', 'user_based': True}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [8]:
!head -10 /root/.surprise_data/ml-100k/ml-100k/u.item

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

# Id to Name Lookup
Let's write a small method that will convert id to name, and name to id

In [9]:
def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

In [10]:
# test this function
rid_to_name, name_to_rid = read_item_names()

In [11]:
rid_to_name["1"]

'Toy Story (1995)'

In [12]:
name_to_rid["Twelve Monkeys (1995)"]

'7'

In [13]:
# Find top 10 movies similar to movie with id 100

movie_inner_id = algo.trainset.to_inner_iid("2")
movie_name = rid_to_name["2"]

# Retrieve inner ids of the nearest neighbors of Toy Story.
movie_neighbors = algo.get_neighbors(movie_inner_id, k=10)

# Convert inner ids of the neighbors into names.
movie_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in movie_neighbors)
movie_neighbors = (rid_to_name[rid]
                       for rid in movie_neighbors)

print()

print('The 10 nearest neighbors of ' + movie_name)
for movie in movie_neighbors:
    print(movie)


The 10 nearest neighbors of GoldenEye (1995)
unknown
My Family (1995)
What Happened Was... (1994)
Flower of My Secret, The (Flor de mi secreto, La) (1995)
Wizard of Oz, The (1939)
Clueless (1995)
Batman & Robin (1997)
I.Q. (1994)
Theodore Rex (1995)
Basketball Diaries, The (1995)


Let's now apply the algorithm and figure out it's accuracy

In [14]:
testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)  # ~ 0.68 (which is low)

RMSE: 0.4774


0.4773574755045231

Now, let's also try some baseline methods. Follow the code available here:

https://github.com/NicolasHug/Surprise/blob/fa7455880192383f01475162b4cbd310d91d29ca/examples/baselines_conf.py

For more elaborate testing and validation, follow steps mentioned here
https://github.com/NicolasHug/Surprise/blob/fa7455880192383f01475162b4cbd310d91d29ca/examples/grid_search_usage.py

# Assignment

In this part, you will use the dataset that is provided along with the following Kaggle competition

https://www.kaggle.com/arashnic/book-recommendation-dataset


I have uploaded the files for you at

Ratings file - https://an-utd-course.s3.us-west-1.amazonaws.com/CompDS/Ratings.csv

Books file - https://an-utd-course.s3.us-west-1.amazonaws.com/CompDS/Books.csv


Follow the steps below to create a recommendation system from this data

In [15]:
# TODO: Read both the data files into Pandas dataframes
ratings = pd.read_csv(" https://an-utd-course.s3.us-west-1.amazonaws.com/CompDS/Ratings.csv")
books = pd.read_csv("https://an-utd-course.s3.us-west-1.amazonaws.com/CompDS/Books.csv")

/tmp/ipython-input-3780184859.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("https://an-utd-course.s3.us-west-1.amazonaws.com/CompDS/Books.csv")


In [21]:
# TODO: Answer the following questions:

# How many ratings and how many books are there in the dataset
ratings.shape

# Find the top 10 books have received the highest count of ratings. You should output the id of the book, its title, and the count of ratings received.

df = ratings.merge(books, on = "ISBN")

df.head()
pd.DataFrame(df.groupby("Book-Title").count()["Book-Rating"].sort_values(ascending=False)).reset_index()

,Book-Title,Book-Rating
0,Wild Animus,2502
1,The Lovely Bones: A Novel,1295
2,The Da Vinci Code,898
3,A Painted House,838
4,The Nanny Diaries: A Novel,828
...,...,...
241066,Ã?Â?ber den ProzeÃ?Â? der Zivilisation 2.,1
241067,Ã?Â?ber den Wassern schweben.,1
241068,Ã?Â?ber den letzten Stand der Dinge.,1
241069,Ã?Â?ber die Freiheit.,1


In [17]:
df.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...


In [18]:
# TODO: Important - You may not be able use the whole dataset for model creation, so you need to create a
# smaller sample to proceeed further
# Here is what I did:
ratings_short = df[['User-ID', 'Book-Title', 'Book-Rating']].sample(n = 1000, random_state = 42)
# you can try larger values of n, if the system allows you.

In [22]:
ratings_short.head()

,User-ID,Book-Title,Book-Rating
770118,208390,Icy Sparks,5
454727,123625,"The Beast from the East (Goosebumps, No 43)",7
71725,16943,Cloud Music,0
535451,144255,The Street Lawyer,4
46502,11676,Paradise,10


In [23]:
# TODO: Use the data to create a custom dataset in the surprise library
# Steps to do this are: https://surprise.readthedocs.io/en/stable/getting_started.html#use-a-custom-dataset
reader = Reader(rating_scale=(1, 10))
dataset1 = Dataset.load_from_df(ratings_short[['User-ID', 'Book-Title', 'Book-Rating']], reader)

trainset = dataset1.build_full_trainset()

In [24]:
for x in trainset.all_items():
  print(trainset.to_raw_iid(x))

Icy Sparks
The Beast from the East (Goosebumps, No 43)
Cloud Music
The Street Lawyer
Paradise
Mooring Against the Tide: Writing Fiction and Poetry
Incredible Journey
Running in the Family (Vintage International)
River, Cross My Heart
In Sylvan Shadows (Forgotten Realms Novel: Cleric Quintet)
Flesh And Blood
The Island Of Heavenly Daze
Zen in the Art of Writing
Gott ist mein Broker.
The Rainmaker
Tis: A Memoir
The City of Joy
To Hatred Turned: A True Story of Love and Death in Texas
The Waste Lands (The Dark Tower, Book 3)
Vinegar Hill (Oprah's Book Club (Paperback))
Bridget Jones's Diary
Beezus and Ramona
Romeo and Juliet (New Folger Library Shakespeare (Paperback))
Firebrand (Mira Historical Romance)
The Stars Shine Down
Le Seigneur des Anneaux: La Communaute de l'Anneau
My Gal Sunday
Rodale's Book of Hints, Tips &amp; Everyday Wisdom
Another Roadside Attraction
Witch Boy
The Scottish Bride (Bride Trilogy (Paperback))
Mondo Barbie
Nicole Brown Simpson: The Private Diary of a Life Inte

In [25]:
# TODO: Choose a book at random and use the KNNBasic algorithm to find out its 10 closest neighbors. Do the results make
# sense?

algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)
book_inner_id = algo.trainset.to_inner_iid("Snow Walker")

book_neighbors = algo.get_neighbors(book_inner_id, k=10)

book_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in book_neighbors)

for book in book_neighbors:
  print(book)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Icy Sparks
The Beast from the East (Goosebumps, No 43)
Cloud Music
The Street Lawyer
Paradise
Mooring Against the Tide: Writing Fiction and Poetry
Incredible Journey
Running in the Family (Vintage International)
River, Cross My Heart
In Sylvan Shadows (Forgotten Realms Novel: Cleric Quintet)


In [26]:
# TODO: Use ParameterGridSearch on the following algorithms and compare their accuracies. You are free to decide
# which specific parameters to use:
# 1. KNNBaseline
# 2. ALS - Baseline
# 3. SGD - Baseline
# 4. SVD
# You should use a cv value of at least 3 and compare the mean accuracy of each of the algorithms
# Comment on whether there is significant differences in the results of the algorithms

from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate

bsl_options = {'method': 'sgd',
               'learning_rate': .001,
               }
algo = BaselineOnly(bsl_options=bsl_options)

from surprise import KNNBasic
algo_knn = KNNBasic()


from surprise import SVD
algo_svd = SVD(n_factors=50)

print("KNN")
cross_validate(algo_knn, dataset1, verbose=True)


print("Gradient Descent")
cross_validate(algo, dataset1, verbose=True)

print("SVD")
cross_validate(algo_svd, dataset1, verbose=True)


KNN
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.8441  4.0152  3.7915  3.9448  3.7542  3.8700  0.0969  
MAE (testset)     3.5612  3.6916  3.5609  3.6429  3.5282  3.5970  0.0606  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Gradient Descent
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of a

{'test_rmse': array([3.89223431, 3.8304255 , 3.81969417, 3.66977606, 3.82189454]),
 'test_mae': array([3.59069854, 3.55179019, 3.47832231, 3.41400962, 3.5330176 ]),
 'fit_time': (0.007972955703735352,
  0.007850885391235352,
  0.007666587829589844,
  0.0072481632232666016,
  0.008667707443237305),
 'test_time': (0.0008020401000976562,
  0.0007326602935791016,
  0.0006527900695800781,
  0.0006771087646484375,
  0.0006763935089111328)}

In [27]:
from surprise import Dataset, SVD
from surprise.model_selection import GridSearchCV

param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(dataset1)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

3.8451913641417192
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
